In [37]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re
import time

# Functions

In [6]:
def scrape_horse_links(date, venue, max_race_no = 14):

    data = [[date, venue]]

    fixture_df = pd.DataFrame(data, columns = ['Date', 'Venue'])
    
    base_url = "https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx"
    http_headers = {"User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"}

    all_links = []

    # loop for date and venue
    for idx, row in fixture_df.iterrows():
        date = row['Date']
        venue = row['Venue']

        print(f'scraping date: {date} @ {venue}')

        # loop for race numbers
        for race_no in range(1, max_race_no + 1):
            params = {
                "RaceDate": date,
                "Racecourse": venue,
                "RaceNo": race_no   
            }

            print(f'scraping race no: {race_no}')
            time.sleep(1)

            # get the response from the url
            response = requests.get(base_url, params = params, headers = http_headers)
            if response.status_code != 200:
                print(f"  Race {race_no}: Failed to fetch (status {response.status_code}). Stopping.")
                break

            soup = BeautifulSoup(response.text, 'html.parser')

            table = soup.find('table', class_ = 'table_bd')
            if not table:
                print('table not found')
                break

            header_row = table.find('tr') 
            headers = [td.get_text(strip = True) for td in header_row.find_all('td')]
            if 'Horse' not in headers:
                print("no 'horse' column found")
                break

            # get position of the horse column
            horse_col_index = headers.index('Horse')

            for row in table.find_all('tr')[1:]:  # skip header row
                cells = row.find_all('td') # get all cells in the row
                if len(cells) > horse_col_index:
                    td = cells[horse_col_index]
                    a_tag = td.find('a', href = True)
                    if a_tag:
                        href = a_tag['href']
                        if href.startswith('/'):
                            href = "https://racing.hkjc.com" + href
                        all_links.append(href) 
    
    return list(set(all_links))



In [26]:
def scraping_races(list, race_date):

    # Initiate header and rows for storage
    header = None
    rows = []

    for href in list:
        full_url = href + '&Option=1'

        try:
            response = requests.get(full_url, timeout = 10)
            print(f'scraping {full_url}')
        except Exception as e:
            print(f'Request exception for {full_url}: {str(e)}')
            continue

        match = re.search(r'([A-Z]\d{3})$', href)
        horse_id = match.group(1) if match else None

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # find origin and age
            origin_label = soup.find('td', string = lambda text : text and 'Country of Origin' in text)
            if origin_label:
                origin_info = origin_label.find_next().find_next().get_text(strip = True)
            else:
                origin_info = 'na'

            # find colour and sex
            colour_label = soup.find('td', string = lambda text : text and 'Colour / Sex' in text)
            if colour_label:
                colour_info = colour_label.find_next().find_next().get_text(strip = True)
            else:
                colour_info = 'na'

            # find import type
            import_label = soup.find('td', string = lambda text : text and 'Import Type' in text)
            if import_label:
                import_info = import_label.find_next().find_next().get_text(strip = True)
            else:
                import_info = 'na'

            # find sire
            sire_label = soup.find('td', string = lambda text : text and 'Sire' in text)
            if sire_label:
                sire_info = sire_label.find_next().find_next().get_text(strip = True)
            else:
                sire_info = 'na'

            # find dam
            dam_label = soup.find('td', string = lambda text : text and 'Dam' in text)
            if dam_label:
                dam_info = dam_label.find_next().find_next().get_text(strip = True)
            else:
                dam_info = 'na'

            # find dam's sire
            dam_sire_label = soup.find('td', string = lambda text : text and 'Dam\'s Sire' in text)
            if dam_sire_label:
                dam_sire_info = dam_sire_label.find_next().find_next().get_text(strip = True)
            else:
                dam_sire_info = 'na'
            
            # find horse per race info
            horse_table = soup.find('table', class_='bigborder')
            if horse_table:
                if header is None:
                    header_row = horse_table.find('tr')
                    header = [th.get_text(strip=True) for th in header_row.find_all('td')]
                    header = header[:-1]
                    header.append('Horse_id')
                    header.append('Origin / Age')
                    header.append('Colour / Sex')
                    header.append('Import type')
                    header.append('Sire')
                    header.append('Dam')
                    header.append('Dam sire')

                for tr in horse_table.find_all('tr')[1:]:
                    cols = [td.get_text(strip=True) for td in tr.find_all('td')]

                    if all(not col.strip() for col in cols):
                        continue
                    first_col = cols[0].strip()
                    if 'Season' in first_col or first_col == 'Overseas':
                        continue

                    # append all info into the rows 
                    cols = cols[:-2]
                    cols.append(horse_id)
                    cols.append(origin_info)
                    cols.append(colour_info)
                    cols.append(import_info)
                    cols.append(sire_info)
                    cols.append(dam_info)
                    cols.append(dam_sire_info)
                    rows.append(cols)
            else:
                print(f'Table not found in {full_url}')


        else:
            print(f"Failed to retrieve {full_url}")

    if not header:
        raise RuntimeError("No data table header found! Check the structure of the page.")

    df = pd.DataFrame(rows, columns=header)
    df['Date'] = pd.to_datetime(df['Date'], format = 'mixed',  dayfirst = True).dt.strftime('%d/%m/%y')
    race_date = pd.to_datetime(race_date, format = 'mixed', dayfirst=True).strftime('%d/%m/%y')

    df = df[df['Date'] == race_date]

    return df


In [38]:
def append_with_new(new_df, old_df):

    old_df = pd.read_csv(old_df)
    df = pd.concat([old_df, new_df])

    current_time = datetime.now().strftime('%Y%m%d')

    file_path = f'../data/race_data/race_data_{current_time}.csv'

    df.to_csv(file_path, index=False)

    return df

# Workflow

In [65]:
links = scrape_horse_links('28/09/2025', 'ST')

scraping date: 28/09/2025 @ ST
scraping race no: 1
scraping race no: 2
scraping race no: 3
scraping race no: 4
scraping race no: 5
scraping race no: 6
scraping race no: 7
scraping race no: 8
scraping race no: 9
scraping race no: 10
scraping race no: 11
scraping race no: 12
table not found


In [66]:
df = scraping_races(links, '28/09/2025')

scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J321&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H483&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K230&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H027&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K004&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H408&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K098&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H161&Option=1
scraping https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J056&Option=1
scraping https://racing.hkjc.com/raci

In [67]:
df

,RaceIndex,Pla.,Date,RC/Track/Course,Dist.,G,RaceClass,Dr.,Rtg.,Trainer,...,Finish Time,Declar.Horse Wt.,Gear,Horse_id,Origin / Age,Colour / Sex,Import type,Sire,Dam,Dam sire
0,053,05,28/09/25,"ST / Turf / ""C+3""",1200,GF,2,7,81,J Size,...,1.08.61,1149,--,J321,AUS / 5,Bay / Gelding,PPG,Starspangledbanner,Fun In Flight,Fusaichi Pegasus
14,049,03,28/09/25,"ST / Turf / ""C+3""",1200,GF,4,4,49,M Newnham,...,1.09.01,1129,B1,H483,AUS / 6,Brown / Gelding,ISG,Written Tycoon,Lady Gracious,Redoute's Choice
36,048,08,28/09/25,ST / AWT,1200,GD,4,3,51,Y S Tsui,...,1.10.13,1025,B/TT,K230,AUS / 5,Bay / Gelding,PP,Prized Icon,Miss Cacciatore,Snitzel
44,047,03,28/09/25,"ST / Turf / ""C+3""",1600,G,5,7,23,J Richards,...,1.34.92,1106,B/TT,H027,AUS / 6,Bay / Gelding,PPG,Spirit Of Boom,Darook Park,Darci Brahma
81,050,03,28/09/25,"ST / Turf / ""C+3""",1400,GF,4,12,49,Y S Tsui,...,1.21.97,1100,H,K004,NZ / 4,Bay / Gelding,PPG,Almanzor,Holly Holy,Dylan Thomas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,048,12,28/09/25,ST / AWT,1200,GD,4,12,49,K L Man,...,1.11.63,1195,CP/TT,H454,AUS / 6,Bay / Gelding,PP,All Too Hard,Nubian,More Than Ready
2138,045,10,28/09/25,ST / AWT,1200,WS,5,12,38,K L Man,...,1.10.67,1151,B,J471,AUS / 5,Bay / Gelding,PPG,Akeed Mofeed,Dream Beauty,Lacryma Cristi
2146,055,14,28/09/25,"ST / Turf / ""C+3""",1600,GF,3,8,67,D Eustace,...,1.35.43,1022,XB-/TT-,K425,IRE / 4,Grey / Gelding,PP,Caravaggio,Isabella,Galileo
2150,055,12,28/09/25,"ST / Turf / ""C+3""",1600,GF,3,7,72,P C Ng,...,1.35.36,1039,B2,H289,NZ / 6,Bay / Gelding,PPG,Derryn,La Petite,Savabeel


In [68]:
new = append_with_new(df, '../data/race_data/race_data.csv')

/var/folders/rn/zdwl9_vx4yz5r87jv7jnztdh0000gn/T/ipykernel_2379/2704290647.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  old_df = pd.read_csv(old_df)


In [69]:
new

,RaceIndex,Pla.,Date,RC/Track/Course,Dist.,G,RaceClass,Dr.,Rtg.,Trainer,...,Finish Time,Declar.Horse Wt.,Gear,Horse_id,Origin / Age,Colour / Sex,Import type,Sire,Dam,Dam sire
0,238,12,10/12/2023,"ST / Turf / ""A""",1600,G,G1,10,--,T Yasuda,...,1.35.46,1187,H,H811,JPN,Bay / Horse,VIS,Just A Way,Epic Love,Dansili
1,623,05,30/04/2023,"ST / Turf / ""A""",2000,G,G1,7,--,T Yasuda,...,2.02.71,1179,H,H811,JPN,Bay / Horse,VIS,Just A Way,Epic Love,Dansili
2,240,02,11/12/2022,"ST / Turf / ""A""",2000,G,G1,6,--,T Yasuda,...,2.00.44,1150,--,H811,JPN,Bay / Horse,VIS,Just A Way,Epic Love,Dansili
3,402,11,06/02/2021,"ST / Turf / ""C""",1200,G,5,2,18,C H Yip,...,1.10.66,1045,CP-/TT-,C017,AUS,Brown / Gelding,PPG,Smart Missile,Pyrography,Danzero
4,296,05,26/12/2020,"ST / Turf / ""A+3""",1200,G,5,14,18,C H Yip,...,1.10.42,1058,CP/TT,C017,AUS,Brown / Gelding,PPG,Smart Missile,Pyrography,Danzero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2112,048,12,28/09/25,ST / AWT,1200,GD,4,12,49,K L Man,...,1.11.63,1195,CP/TT,H454,AUS / 6,Bay / Gelding,PP,All Too Hard,Nubian,More Than Ready
2138,045,10,28/09/25,ST / AWT,1200,WS,5,12,38,K L Man,...,1.10.67,1151,B,J471,AUS / 5,Bay / Gelding,PPG,Akeed Mofeed,Dream Beauty,Lacryma Cristi
2146,055,14,28/09/25,"ST / Turf / ""C+3""",1600,GF,3,8,67,D Eustace,...,1.35.43,1022,XB-/TT-,K425,IRE / 4,Grey / Gelding,PP,Caravaggio,Isabella,Galileo
2150,055,12,28/09/25,"ST / Turf / ""C+3""",1600,GF,3,7,72,P C Ng,...,1.35.36,1039,B2,H289,NZ / 6,Bay / Gelding,PPG,Derryn,La Petite,Savabeel
